In [88]:
import pandas as pd
import json
import random

In [89]:
success_df = pd.read_pickle("merged.df")

In [90]:
import pandas as pd
import uuid

# Assuming the DataFrame is already defined as df

# Get unique genotypes from the 'genotype' column
unique_genotypes = success_df['genotype'].explode().unique().tolist()

# Create a dictionary to store the genotype ID pairs
genotype_id_dict = {}

# Assign a unique ID to each genotype
for genotype in unique_genotypes:
    genotype_id = str(uuid.uuid4())
    genotype_id_dict[genotype] = genotype_id

In [91]:
success_df.at[0, 'index_dict']

{'WT': [(151, 153), (612, 614)], 'KO': [(193, 195), (638, 640)]}

In [92]:
import pandas as pd
import json
import random

# Assuming the DataFrame is already defined as df

# Generate unique IDs for each row
success_df['id'] = [str(random.randint(100000, 999999)) for _ in range(len(success_df))]

# Create a new DataFrame with the desired columns
new_df = success_df[['id', 'Project ID', 'Project Name', 'Title', 'Description', 'genotype', 'index_dict']].copy()

# Rename the columns to match the desired JSON format
new_df.columns = ['id', 'project_id', 'project_name', 'project_title', 'content', 'genotype', 'annotations']

# Convert the genotype column to a list of dictionaries
new_df['genotype'] = new_df['genotype'].apply(lambda x: [{'id': genotype_id_dict[g], 'value': g} for g in x])

# Convert the index_dict column to a list of dictionaries
new_df['annotations'] = new_df['annotations'].apply(lambda x: [{'start': g, 'end': a, 'tag_name': 'GENOTYPE', 'value': k} for k, v in x.items() for g, a in v if v])

# Convert the DataFrame to JSON
json_data = new_df.to_json(orient='records')

# Parse the JSON string
parsed_data = json.loads(json_data)

# Create a dictionary with the 'data' key and the parsed data
result = {'data': parsed_data}

In [93]:
result['data'][2]

{'id': '265085',
 'project_id': 'PRJNA986078',
 'project_name': 'Evi1 governs Kdm6b-mediated histone demethylation to finely regulate the Laptm4b-driven mTORC pathway in vivo [LK]',
 'project_title': 'Evi1 governs Kdm6b-mediated histone demethylation to finely regulate the Laptm4b-driven mTORC pathway in vivo [LK]',
 'content': 'Ecotropic viral integration site 1 (EVI1/MECOM) overexpression is common in myeloid malignancies. We present a new Evi1 transgenic mouse model with inducible expression in hematopoietic stem cells (HSCs) and progenitor cells (HPCs) at lower levels. Upon exogenous Evi1 induction, mice displayed anemia, thrombocytopenia, lymphopenia, and dysplasia in erythroid and megakaryocyte cells with a significant expansion of committed myeloid progenitor cells, resembling human Myelodysplastic syndrome/Myeloproliferative neoplasm (MDS/MPN)-like disease. Methods: Lin-C-Kit+ cells were isolated from 3 pairs of WT and Evi1 overexpressing mice. Around 1X106 Lin-C-Kit+ cells wer

In [94]:
with open('data.json', 'w') as f:
    json.dump(result, f)

In [95]:
with open('data.json', 'r') as f:
    data = json.load(f)
    
print(data['data'][0])

{'id': '944377', 'project_id': 'PRJNA986528', 'project_name': 'PIKfyve controls dendritic cell function and tumor immunity I', 'project_title': 'PIKfyve controls dendritic cell function and tumor immunity I', 'content': 'To understand the mechanism of Pikfyve loss in dendritic cells, we cultured conventional dendritic cells isolated from the bone marrow of Pikfyvef/f (“WT”) and mice with ItgaxTg/0 Pikfyvef/f (“KO”) mice and performed transcriptome profiling on RNA extracted from cells on Day 9 in culture. Taken together, these data demonstrate the PIKfyve mediates suppression of DC transcriptional maturation programs through the alternate/non-canonical NF-κB regulatory pathway. Overall design: Comparative gene expression profiling of RNA-seq data from primary cultured conventional dendritic cells isolated from wild-type (WT) or Pikfyve-deficient (KO) mice (littermate pair-matched biological triplicates).', 'genotype': [{'id': 'd5ffd8e8-0229-469b-ba10-1462f309718b', 'value': 'WT'}, {'id

In [96]:

training_data = {'classes' : ['GENOTYPE'], 'annotations' : []}
for example in data['data']:
    temp_dict = {}
    temp_dict['text'] = example['content']
    temp_dict['entities'] = []
    for annotation in example['annotations']:
        start = annotation['start']
        end = annotation['end']
        label = annotation['tag_name'].upper()
        temp_dict['entities'].append((start, end, label))
    training_data['annotations'].append(temp_dict)
  
print(training_data['annotations'][0])

{'text': 'To understand the mechanism of Pikfyve loss in dendritic cells, we cultured conventional dendritic cells isolated from the bone marrow of Pikfyvef/f (“WT”) and mice with ItgaxTg/0 Pikfyvef/f (“KO”) mice and performed transcriptome profiling on RNA extracted from cells on Day 9 in culture. Taken together, these data demonstrate the PIKfyve mediates suppression of DC transcriptional maturation programs through the alternate/non-canonical NF-κB regulatory pathway. Overall design: Comparative gene expression profiling of RNA-seq data from primary cultured conventional dendritic cells isolated from wild-type (WT) or Pikfyve-deficient (KO) mice (littermate pair-matched biological triplicates).', 'entities': [(151, 153, 'GENOTYPE'), (612, 614, 'GENOTYPE'), (193, 195, 'GENOTYPE'), (638, 640, 'GENOTYPE')]}


In [97]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [98]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

  0%|▏                                         | 6/1804 [00:00<00:37, 48.25it/s]

WT 151 153 GENOTYPE
WT 612 614 GENOTYPE
KO 193 195 GENOTYPE
KO 638 640 GENOTYPE
Control 1481 1488 GENOTYPE
Hdac1/Hdac2 double mutant 1493 1518 GENOTYPE
WT 603 605 GENOTYPE
WT 1001 1003 GENOTYPE
Evi1 overexpressing 610 629 GENOTYPE
Evi1 overexpressing 913 932 GENOTYPE
Evi1 overexpressing 1008 1027 GENOTYPE
wild-type 1327 1336 GENOTYPE
Control 1999 2006 GENOTYPE
PGAM5 KD 2010 2018 GENOTYPE
ROSA26-eYFP+/+ Cdh5-CreERT2+ 2144 2172 GENOTYPE
ROSA26-eYFP+/+ Cdh5-CreERT2+ 2144 2172 GENOTYPE
Prkci flox/flox 1648 1663 GENOTYPE
Prkci flox/flox 1728 1743 GENOTYPE
MC4R-KO 881 888 GENOTYPE
Control 1833 1840 GENOTYPE
Control 1881 1888 GENOTYPE
PFC_D2R-KO 1636 1646 GENOTYPE
PFC_D2R-KO 1959 1969 GENOTYPE
PFC_D2R-KO 2009 2019 GENOTYPE
Zfp423GFP/+ 103 114 GENOTYPE
WT 57 59 GENOTYPE
iKO 66 69 GENOTYPE


  1%|▍                                        | 20/1804 [00:00<00:28, 62.02it/s]

GR deletion 1984 1995 GENOTYPE
GR deletion 2302 2313 GENOTYPE
GR deletion 2533 2544 GENOTYPE
GR deletion 2620 2631 GENOTYPE
Loxp/Loxp 145 154 GENOTYPE
Loxp/Loxp 145 154 GENOTYPE
Loxp/Loxp 145 154 GENOTYPE
wild-type 1154 1163 GENOTYPE
nmrHAS2 355 362 GENOTYPE
nmrHAS2 365 372 GENOTYPE
nmrHAS2 564 571 GENOTYPE
nmrHAS2 660 667 GENOTYPE
nmrHAS2 1262 1269 GENOTYPE
nmrHAS2 1365 1372 GENOTYPE
Lgr5CreER/ KrasLSL-G12D/p53fl/fl/Rosa26-YFP+/+ 1757 1803 GENOTYPE
Lgr5CreER/ KrasLSL-G12D/p53fl/fl/Rosa26-YFP+/+ 1763 1809 GENOTYPE
WT 1659 1661 GENOTYPE
FOXO3 KO 1619 1627 GENOTYPE
wildtype 2340 2348 GENOTYPE
tumor 119 124 GENOTYPE
None 1872 1877 GENOTYPE
Skipping entity
None 1985 1990 GENOTYPE
Skipping entity
tumor 2259 2264 GENOTYPE
None 2763 2768 GENOTYPE
Skipping entity
Retnla overexpression 1957 1978 GENOTYPE
bigenic 1652 1659 GENOTYPE
bigenic 1827 1834 GENOTYPE
bigenic 1895 1902 GENOTYPE
monogenic 1669 1678 GENOTYPE
monogenic 2001 2010 GENOTYPE
None 229 233 GENOTYPE
Skipping entity
None 433 437 GEN

  4%|█▋                                      | 78/1804 [00:00<00:10, 172.37it/s]

...

B6 control 249 259 GENOTYPE
wild type 645 654 GENOTYPE
Dsg2-W2A homozygouse mutant 664 691 GENOTYPE
Y334C MT 2163 2171 GENOTYPE
C57BL6/J 241 249 GENOTYPE
C57BL6/J 301 309 GENOTYPE
TauPS2APP 189 198 GENOTYPE
TauPS2APP 885 894 GENOTYPE
PS2APP 141 147 GENOTYPE
None 192 198 GENOTYPE
Skipping entity
PS2APP 867 873 GENOTYPE
None 888 894 GENOTYPE
Skipping entity
CTCF-K20R 388 397 GENOTYPE
wild type 374 383 GENOTYPE
Parva-fl/fl*hNPHS2Cre 43 64 GENOTYPE
Parva-fl/fl*hNPHS2Cre 381 402 GENOTYPE
Parva-fl/ 43 54 GENOTYPE
Parva-fl/ 381 392 GENOTYPE
WT 170 172 GENOTYPE
WT 221 223 GENOTYPE
RUNX1-RUNX1T1 94 107 GENOTYPE
WT 1659 1661 GENOTYPE
HSC-Spred1 wt CML mice 2550 2572 GENOTYPE
Spred1 wt CML mice 2554 2572 GENOTYPE
Spred1 wt CML mice 2628 2646 GENOTYPE
Spred1 wt CML mice 2703 2721 GENOTYPE
wild type 1025 1034 GENOTYPE
ARF-null 278 286 GENOTYPE
wild type 260 269 GENOTYPE
wild type 339 348 GENOTYPE
wild type 434 443 GENOTYPE
NRAS G12D 406 415 GENOTYPE
wild type 1130 1139 GENOTYPE
wild type 4831 4840 

 97%|████████████████████████████████████▉ | 1755/1804 [00:05<00:00, 347.16it/s]

WT 467 469 GENOTYPE
WT 518 520 GENOTYPE
WT 856 858 GENOTYPE
Siglec-H +/+ 394 406 GENOTYPE
Siglec-H -/- 423 435 GENOTYPE
p53- 296 300 GENOTYPE
p53- 883 887 GENOTYPE
p53- 1578 1582 GENOTYPE
p53- 1735 1739 GENOTYPE
p53+ 334 338 GENOTYPE
p53+ 438 442 GENOTYPE
p53+ 677 681 GENOTYPE
p53+ 1033 1037 GENOTYPE
p53+ 1294 1298 GENOTYPE
p53+ 1616 1620 GENOTYPE
p53+ 1768 1772 GENOTYPE
WT 1135 1137 GENOTYPE
Cre 4 7 GENOTYPE
Cre 147 150 GENOTYPE
Cre 195 198 GENOTYPE
Cre 260 263 GENOTYPE
Cre 304 307 GENOTYPE
Cre 439 442 GENOTYPE
None 476 479 GENOTYPE
Skipping entity
None 616 619 GENOTYPE
Skipping entity
None 858 861 GENOTYPE
Skipping entity
None 975 978 GENOTYPE
Skipping entity
Cre 1040 1043 GENOTYPE
None 1167 1170 GENOTYPE
Skipping entity
None 1172 1175 GENOTYPE
Skipping entity
WT 1125 1127 GENOTYPE
Cre 4 7 GENOTYPE
Cre 147 150 GENOTYPE
Cre 195 198 GENOTYPE
Cre 260 263 GENOTYPE
Cre 304 307 GENOTYPE
None 460 463 GENOTYPE
Skipping entity
Cre 532 535 GENOTYPE
None 746 749 GENOTYPE
Skipping entity
Cre 103

100%|██████████████████████████████████████| 1804/1804 [00:06<00:00, 297.52it/s]


KC 706 708 GENOTYPE
KC 834 836 GENOTYPE
KC 902 904 GENOTYPE
KC 1165 1167 GENOTYPE
KC 1177 1179 GENOTYPE
KC 1375 1377 GENOTYPE
KC 1393 1395 GENOTYPE
KC 2053 2055 GENOTYPE
KC 2070 2072 GENOTYPE
KC/PD 2070 2075 GENOTYPE
Dusp1-/- 665 673 GENOTYPE
Dusp1-/- 723 731 GENOTYPE
Dusp1-/- 932 940 GENOTYPE
Dusp1-/- 1469 1477 GENOTYPE
wild type 1450 1459 GENOTYPE
p53 wt 1237 1243 GENOTYPE
wild-type 216 225 GENOTYPE
wild-type 279 288 GENOTYPE
wild-type 697 706 GENOTYPE
Dusp6 knockout 711 725 GENOTYPE
Nix-/-Bnip3-/- 1161 1175 GENOTYPE
WT 706 708 GENOTYPE
Alk5iEKO 1083 1091 GENOTYPE
wild type 1358 1367 GENOTYPE
MAVS -/- 350 358 GENOTYPE
WT 84 86 GENOTYPE
wild type 921 930 GENOTYPE
HdhQ7/Q111 (KI) 2308 2323 GENOTYPE
HdhQ7/7 (wild-type) 2284 2303 GENOTYPE
Ino80 cKO 1568 1577 GENOTYPE
control 400 407 GENOTYPE
control 1429 1436 GENOTYPE
control 347 354 GENOTYPE
None 782 789 GENOTYPE
Skipping entity
control 890 897 GENOTYPE
Acsl6-/- 902 910 GENOTYPE
Adcy10-/- 504 513 GENOTYPE
Adcy10-/- 767 776 GENOTYPE
Adcy

In [99]:
doc.ents

(WT, LIPTER KO, WT)

In [115]:
colors = {"GENOTYPE": "#F67DE3"}
options = {"colors": colors}

# Convert the generator object to a list
doc_list = list(doc_bin.get_docs(nlp.vocab))
doc = doc_list[9]

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)